In [ ]:
# FAISS ( Facebook AI Similarity Search) is a library for efficient similarity search and clustering of dense vectors.
# It is particularly useful for applications like nearest neighbor search in high-dimensional spaces.
# It contains algorithm that searches in set of vectors of any size, up to billions of vectors.

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [10]:
loader = TextLoader("text.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1281, which is longer than the specified 1000


In [11]:
docs

[Document(metadata={'source': 'text.txt'}, page_content='Readme'),
 Document(metadata={'source': 'text.txt'}, page_content="Sagemaker integration\nWhen SageMaker starts a container, it will invoke the container with an argument of either\xa0train\xa0or\xa0serve. We have set this container to accept the\xa0serve\xa0command which will, in turn, run our application.\n* serve: The wrapper that starts the inference server. In most cases, you can use this file as-is.\n* wsgi.py: The start up shell for the individual server workers. This only needs to be changed if you changed where predict.py is located or is named.\n* predict.py: The algorithm-specific inference server. This is the file that you modify with your own algorithm's code.\n* nginx.conf: Now included within the mdf-base-image. The configuration for the nginx master server that manages the multiple workers.\nMock Interview Transcript – NPTB Recommender System\n🧑\u200d💼 Interviewer: Hi Prathmesh, thanks for joining. Let’s start wit

In [13]:
embeds = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs, embeds)


In [14]:
query = "How was the data gathered?"
docs = db.similarity_search(query)
docs

[Document(id='f44c3015-d6f3-4c1d-a1e1-879da5c20e27', metadata={'source': 'text.txt'}, page_content='🧑\u200d💼 Interviewer: What were some of the technical challenges in this project?\n🧑\u200d💻 Prathmesh: One challenge was managing memory usage and load time—since the model was 23GB, it took around 160 seconds to load initially. We used warm-started containers and lazy loading strategies to minimize latency. Handling the sparsity of item combinations was another challenge, which we addressed by using product-level fallbacks for recommendations.\n\n\nTo deal with sparsity in SKU-based association rules, we created a fallback mechanism. If a user’s basket had no matching SKU-level rules, we mapped those SKUs to their product categories or descriptions and used a secondary set of association rules trained at that level. This allowed us to preserve recommendation quality even in edge cases.”'),
 Document(id='aa6f0617-b73c-4f7e-a062-0f416f4137ab', metadata={'source': 'text.txt'}, page_content

In [15]:
docs[0].page_content

'🧑\u200d💼 Interviewer: What were some of the technical challenges in this project?\n🧑\u200d💻 Prathmesh: One challenge was managing memory usage and load time—since the model was 23GB, it took around 160 seconds to load initially. We used warm-started containers and lazy loading strategies to minimize latency. Handling the sparsity of item combinations was another challenge, which we addressed by using product-level fallbacks for recommendations.\n\n\nTo deal with sparsity in SKU-based association rules, we created a fallback mechanism. If a user’s basket had no matching SKU-level rules, we mapped those SKUs to their product categories or descriptions and used a secondary set of association rules trained at that level. This allowed us to preserve recommendation quality even in edge cases.”'

In [16]:
retrieve = db.as_retriever()
retrieve.invoke(query)

[Document(id='f44c3015-d6f3-4c1d-a1e1-879da5c20e27', metadata={'source': 'text.txt'}, page_content='🧑\u200d💼 Interviewer: What were some of the technical challenges in this project?\n🧑\u200d💻 Prathmesh: One challenge was managing memory usage and load time—since the model was 23GB, it took around 160 seconds to load initially. We used warm-started containers and lazy loading strategies to minimize latency. Handling the sparsity of item combinations was another challenge, which we addressed by using product-level fallbacks for recommendations.\n\n\nTo deal with sparsity in SKU-based association rules, we created a fallback mechanism. If a user’s basket had no matching SKU-level rules, we mapped those SKUs to their product categories or descriptions and used a secondary set of association rules trained at that level. This allowed us to preserve recommendation quality even in edge cases.”'),
 Document(id='aa6f0617-b73c-4f7e-a062-0f416f4137ab', metadata={'source': 'text.txt'}, page_content

In [17]:
docs_score = db.similarity_search_with_score(query)
docs_score

[(Document(id='f44c3015-d6f3-4c1d-a1e1-879da5c20e27', metadata={'source': 'text.txt'}, page_content='🧑\u200d💼 Interviewer: What were some of the technical challenges in this project?\n🧑\u200d💻 Prathmesh: One challenge was managing memory usage and load time—since the model was 23GB, it took around 160 seconds to load initially. We used warm-started containers and lazy loading strategies to minimize latency. Handling the sparsity of item combinations was another challenge, which we addressed by using product-level fallbacks for recommendations.\n\n\nTo deal with sparsity in SKU-based association rules, we created a fallback mechanism. If a user’s basket had no matching SKU-level rules, we mapped those SKUs to their product categories or descriptions and used a secondary set of association rules trained at that level. This allowed us to preserve recommendation quality even in edge cases.”'),
  np.float32(2844.3572)),
 (Document(id='aa6f0617-b73c-4f7e-a062-0f416f4137ab', metadata={'source

In [18]:
em_vec = embeds.embed_query(query)


In [19]:
db.similarity_search_by_vector(em_vec)

[Document(id='f44c3015-d6f3-4c1d-a1e1-879da5c20e27', metadata={'source': 'text.txt'}, page_content='🧑\u200d💼 Interviewer: What were some of the technical challenges in this project?\n🧑\u200d💻 Prathmesh: One challenge was managing memory usage and load time—since the model was 23GB, it took around 160 seconds to load initially. We used warm-started containers and lazy loading strategies to minimize latency. Handling the sparsity of item combinations was another challenge, which we addressed by using product-level fallbacks for recommendations.\n\n\nTo deal with sparsity in SKU-based association rules, we created a fallback mechanism. If a user’s basket had no matching SKU-level rules, we mapped those SKUs to their product categories or descriptions and used a secondary set of association rules trained at that level. This allowed us to preserve recommendation quality even in edge cases.”'),
 Document(id='aa6f0617-b73c-4f7e-a062-0f416f4137ab', metadata={'source': 'text.txt'}, page_content

In [20]:
# saving the vectore store to disk
db.save_local("faiss_index")